In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from sklearn.metrics import plot_roc_curve
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np



In [2]:
def HeaderTrain():
    
    
    
    
    
    
    df = pd.read_csv('preprocessed_spam_ham_phishing.csv')

    df = df[df['label'] != 2]
 
    
    df['label'].value_counts()

    feature_list = ['time_zone',
    'str_content-type_texthtml',
    'str_to_chevron',
    'domain_match_from_return-path',
    'missing_importance',
    'missing_x-mailer',
    'date_comp_date_received',
    'str_return-path_bounce',
    'missing_user-agent',
    'length_from',
    'missing_thread-index',
    'missing_mime-version',
    'domain_val_message-id',
    'str_from_question',
    'str_from_chevron',
    'domain_match_message-id_from',
    'missing_domainkey-signature',
    'missing_x-mailing-list',
    'domain_match_message-id_return-path',
    'missing_content-disposition',
    'missing_x-mailman-version',
    'domain_match_to_from',
    'missing_list-unsubscribe',
    'domain_match_errors-to_from',
    'span_time',
    'domain_match_message-id_reply-to',
    'content-length',
    'lines',
    'day_of_week',
    'missing_precedence',
    'domain_match_errors-to_message-id',
    'missing_reply-to',
    'domain_match_sender_from',
    'missing_mailing-list',
    'received_str_forged',
    'str_precedence_list',
    'domain_match_to_received',
    'missing_x-spam-status',
    'missing_content-type',
    'content-encoding-val',
    'domain_match_errors-to_reply-to',
    'missing_received-spf',
    'missing_references',
    'domain_match_to_message-id',
    'missing_x-original-to',
    'label']

    df = df[feature_list]
    
    df_Y = df['label']
    df_X = df.drop('label', axis=1)
    
  

    features_list = df_X.columns
    
    
    
    
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()

    scaler.fit(df_X)
    df_X = scaler.transform(df_X)
    df_X = pd.DataFrame(df_X, columns=features_list)
    
    
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(df_X,df_Y,test_size = 0.75, random_state = 42)
    
    
    first_col = X_test.iloc[:,:1]
    
    
    
    from sklearn.ensemble import StackingClassifier

    '''
    base_learners = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                    ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                    ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                    ('svm', SVC(C=10, kernel='rbf', tol=0.001))]
    '''

    base_learners_set1 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                    ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                    ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance'))]

    base_learners_set2 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                    ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')), 
                    ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

#     base_learners_set3 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
#                     ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
#                     ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

#     base_learners_set4 = [('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
#                     ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
#                     ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

    base_learners = []
    base_learners.append(base_learners_set1)
    base_learners.append(base_learners_set2)
#     base_learners.append(base_learners_set3)
#     base_learners.append(base_learners_set4)

    for base_learner_group in base_learners:

        meta_learner = LogisticRegression()

        clf = StackingClassifier(estimators=base_learner_group, final_estimator=meta_learner)

        # Train the stacked model on the full training data
        clf.fit(X_train, y_train)

        predictions = clf.predict(X_test)

        # Get the evaluation metrics
        print('Accuracy:', accuracy_score(y_test, predictions)*100)
        print('F1 Score:', f1_score(y_test, predictions)*100)
        print('Recall:', recall_score(y_test, predictions)*100)
        print('Precision:', precision_score(y_test, predictions)*100)
        print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
        print('Confusion Matrix:', confusion_matrix(y_test, predictions))
        print('-----------------------------------------\n')
        
        
        

In [3]:
from email.parser import HeaderParser
import pandas as pd
import shutil
from os import listdir, mkdir, path
from pathlib import Path
import sys
import collections
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
from collections import Counter
parser = HeaderParser()
list_of_rows = []

In [4]:
def HeaderInput(textfilename):
    
    
    
    try:

        columns = ['received1',
        'received2',
        'received3',
        'received4',
        'received5',
        'received6',
        'received7',
        'received8',
        'hops',
        'subject',
        'date',
        'message-id',
        'from',
        'return-path',
        'to',
        'content-type',
        'mime-version',
        'x-mailer',
        'content-transfer-encoding',
        'x-mimeole',
        'x-priority',
        'list-id',
        'lines',
        'x-virus-scanned',
        'status',
        'content-length',
        'precedence',
        'delivered-to',
        'list-unsubscribe',
        'list-subscribe',
        'list-post',
        'list-help',
        'x-msmail-priority',
        'x-spam-status',
        'sender',
        'errors-to',
        'x-beenthere',
        'list-archive',
        'reply-to',
        'x-mailman-version',
        'x-miltered',
        'x-uuid',
        'x-virus-status',
        'x-spam-level',
        'x-spam-checker-version',
        'references',
        'in-reply-to',
        'user-agent',
        'thread-index',
        'cc',
        'received-spf',
        'x-original-to',
        'content-disposition',
        'mailing-list',
        'x-spam-check-by',
        'domainkey-signature',
        'importance',
        'x-mailing-list',
        'label']




        def addEmailsToDict(email_list):
            global parser, list_of_rows

            # The label for phishing, set to '2'
            label = 2


            # Read the full email content
            for email in email_list:
                row_dict = {}

                # Parse the email content
                h = parser.parsestr(email)

                # Parse recieved field
                received_list = h.get_all('received')
                hops = 0
                if received_list is not None:
                    hops = len(received_list)
                    col_name_recieved = 'received'

                    for inx, received_field in enumerate(received_list):
                        col = col_name_recieved + str(inx+1)
                        row_dict[col] = received_field


                # Make everything lowercase to avoid issues
                features_lower_case = [x.lower() for x in h.keys()]

                # Parse everything else
                new_row = dict(zip(features_lower_case, h.values()))
                new_row['hops'] = hops


                for key,value in new_row.items():
                    if key in columns:
                        row_dict['label'] = label
                        row_dict[key] = value


                list_of_rows.append(row_dict)

        def main():
            global list_of_rows

            file_path = textfilename

            # Read the full email content
            emailStr = ''
            try:
                with open(file_path, encoding='latin_1') as emailFile:
                    for line in emailFile:
                        emailStr += line
            except UnicodeDecodeError:
                print('Unicode Error!')

            email_list = emailStr.split('\nFrom jose@monkey.org')
            #print(email_list[1])

            addEmailsToDict(email_list)

            # Create the dataframe
            df = pd.DataFrame(list_of_rows)

            final_columns = []
            for col in columns:
                if col in df.columns:
                    final_columns.append(col)

            df = df[final_columns]

            # Output the dataframe to a .csv file
            df.to_csv('spamTest_out_2021.csv', index=False)

        if __name__ == '__main__':
            main()





        FILE_NAME2021 = 'spamTest_out_2021.csv'

        df = pd.read_csv(FILE_NAME2021)

        for col in df.columns:
            if col not in df.columns:
                df[col] = np.nan


        initial_features_list = ['received1', 'received2', 'received3', 'received4', 'received5',
           'received6', 'received7', 'received8', 'received9', 'received10',
           'received11', 'received12', 'received13', 'received14',
           'received15', 'received16', 'subject', 'date',
           'message-id', 'from', 'return-path', 'to', 'content-type',
           'mime-version', 'x-mailer', 'content-transfer-encoding',
           'x-mimeole', 'x-priority', 'list-id', 'lines', 'x-virus-scanned',
           'status', 'content-length', 'precedence', 'delivered-to',
           'list-unsubscribe', 'list-subscribe', 'list-post', 'list-help',
           'x-msmail-priority', 'x-spam-status', 'sender', 'errors-to',
           'x-beenthere', 'list-archive', 'reply-to', 'x-mailman-version',
           'x-miltered', 'x-uuid', 'x-virus-status', 'x-spam-level',
           'x-spam-checker-version', 'references', 'in-reply-to',
           'user-agent', 'thread-index', 'cc', 'received-spf',
           'x-original-to', 'content-disposition', 'mailing-list',
           'x-spam-check-by', 'domainkey-signature', 'importance',
           'x-mailing-list']

        label_name = 'label'

        final_features_list = ['hops']




        missing_feature_names = []

        for name in initial_features_list:
            missing_feature_names.append('missing_'+name)

        for feature, name in zip(initial_features_list, missing_feature_names):
            if feature in df:
                df.loc[df[feature].isnull(),name] = 1
                df.loc[-df[feature].isnull(), name] = 0

        final_features_list.extend(missing_feature_names)


        df = df.replace(np.nan, '', regex=True)



        df[~(df['content-transfer-encoding'].str.contains('(?i)8bit')) & ~(df['content-transfer-encoding'].str.contains('(?i)7bit'))]['label'].value_counts()


        import re

        def content_encoding_val(row):
          if 'content-transfer-encoding' in row:
              val = row['content-transfer-encoding']
              re1 = re.compile(r'(?i)8bit')
              re2 = re.compile(r'(?i)7bit')
              if (not re1.search(val)) and (not re2.search(val)):
                return 1
              else:
                return 0


        df['content-encoding-val'] = df.apply(content_encoding_val, axis=1)


        final_features_list.append('content-encoding-val')



        def check_received_forged(row):
            num_iters = row['hops']
            col_name_base = 'reveived'

            for i in range(1,num_iters+1):
                if col_name_base + str(i) in row:
                    curr_val = row[col_name_base + str(i)]
                    if 'forged' in curr_val:
                        return 1
                    else:
                        continue

            return 0

        df['received_str_forged'] = df.apply(check_received_forged, axis=1)
        final_features_list.append('received_str_forged')


        df[df['received_str_forged'] == 1]['label'].value_counts()



        def str_based_features_add(old_col_name, new_col_names, items_to_check):

          for col_name, item_to_check in zip(new_col_names, items_to_check):
            if item_to_check == '':
              df.loc[(df[old_col_name].str.fullmatch(item_to_check)), col_name] = 1
              df.loc[~(df[old_col_name].str.fullmatch(item_to_check)), col_name] = 0
            else:
              df.loc[(df[old_col_name].str.contains('(?i)' + item_to_check)), col_name] = 1
              df.loc[~(df[old_col_name].str.contains('(?i)' + item_to_check)), col_name] = 0

          final_features_list.extend(new_col_names)



        # Content-Transfer-Encoding
        new_col_names = ['str_content-encoding_empty']
        items_to_check = ['']
        feature = 'content-transfer-encoding'
        str_based_features_add(feature, new_col_names, items_to_check)

        # From
        new_col_names = ['str_from_question', 'str_from_exclam', 'str_from_chevron']
        items_to_check = ['\?', '!', '<.+>']
        feature = 'from'
        str_based_features_add(feature, new_col_names, items_to_check)

        # To
        new_col_names = ['str_to_chevron', 'str_to_undisclosed', 'str_to_empty']
        items_to_check = ['<.+>', 'Undisclosed Recipients', '']
        feature = 'to'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Message-ID
        new_col_names = ['str_message-ID_dollar']
        items_to_check = ['\$']
        feature = 'message-id'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Return-Path
        new_col_names = ['str_return-path_bounce', 'str_return-path_empty']
        items_to_check = ['bounce', '']
        feature = 'return-path'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Reply-To
        new_col_names = ['str_reply-to_question']
        items_to_check = ['\?']
        feature = 'reply-to'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Received-SPF
        new_col_names = ['str_received-SPF_bad', 'str_received-SPF_softfail', 
                         'str_received-SPF_fail']
        items_to_check = ['bad', 'softfail', 'fail']
        feature = 'received-spf'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Content-Type
        new_col_names = ['str_content-type_texthtml']
        items_to_check = ['text/html']
        feature = 'content-type'
        str_based_features_add(feature, new_col_names, items_to_check)

        # Precedence
        new_col_names = ['str_precedence_list']
        items_to_check = ['list']
        feature = 'precedence'
        str_based_features_add(feature, new_col_names, items_to_check)


        df[df['hops'] > 5]['label'].value_counts()
        df['hops'] = df['hops'].apply(lambda x: 0 if x <= 2 else 1 if x <= 5 else 2)


        def count_chars(field_names, new_col_names):
          for field_name, new_col_name in zip(field_names, new_col_names):
            df[new_col_name] = df[field_name].str.len()

          final_features_list.extend(new_col_names)



        fields_to_find_lengths = ['from']
        new_col_names_lengths = []

        for val in fields_to_find_lengths:
          new_col_names_lengths.append('length_' + val)

        count_chars(fields_to_find_lengths, new_col_names_lengths)


        df[df['length_from'] > 40]['label'].value_counts()

        df['length_from'] = df['length_from'].apply(lambda x: 0 if x > 40 else 1)


        import re

        #https://stackoverflow.com/questions/42407785/regex-extract-email-from-strings

        if 'to' in df:
            df['num_recipients_to'] = df.apply(lambda x: len(re.findall(
                r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x['to'])), axis=1)

        if 'cc' in df:
            df['num_recipients_cc'] = df.apply(lambda x: len(re.findall(
                r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x['cc'])), axis=1)

        if 'from' in df:

            df['num_recipients_from'] = df.apply(lambda x: len(re.findall(
                r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x['from'])), axis=1)

        final_features_list.append('num_recipients_to')
        final_features_list.append('num_recipients_cc')
        final_features_list.append('num_recipients_from')


        df[df['num_recipients_to'] == 0]['label'].value_counts()


        df['num_recipients_to'] = df['num_recipients_to'].apply(lambda x: 0 if x == 0 else 1 if x == 1 else 2)

        if 'num_recipients_cc' in df:

            df['num_recipients_cc'] = df['num_recipients_cc'].apply(lambda x: 0 if x == 0 else 1 if x == 1 else 2)

        df[df['num_recipients_from'] == 0]['label'].value_counts()

        df['num_recipients_from'] = df['num_recipients_from'].apply(lambda x: 0 if x == 0 else 1 if x == 1 else 2)



        import re

        def extract_num_replies(row):
          references_val = row['references']
          all = re.findall(r'<([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)>', 
                          references_val)
          return len(all)


        df['number_replies'] = df.apply(extract_num_replies, axis=1)

        df[df['number_replies'] >= 1]['label'].value_counts()

        df['number_replies'] = df['number_replies'].apply(lambda x: 0 if x >= 1 else 1)

        final_features_list.append('number_replies')

        from dateutil.parser import parse
        from datetime import datetime, timedelta
        import email.utils


        def extract_time_zone(row):
          time_zone = email.utils.parsedate_tz(row['date'])
          if time_zone is None:
            return 'NA'
          else:
            return int(time_zone[9] / (60*60)) % 24


        df['time_zone'] = df.apply(extract_time_zone, axis=1)


        df.loc[df['time_zone'].astype(str).str.contains('NA'), 'time_zone'] = df['time_zone'].value_counts().index[0]


        # Test
        d1 = email.utils.parsedate_tz('Mon, 9 Apr 2007 14:31:03 02300')
        d2 = email.utils.parsedate_tz('Mon, 8 Apr 2007 14:31:03 -0100')

        print((email.utils.mktime_tz(d2)) - (email.utils.mktime_tz(d1)))

        df.loc[df['time_zone'] != 20, 'time_zone'] = 0
        df.loc[df['time_zone'] == 20, 'time_zone'] = 1

        final_features_list.append('time_zone')

        df['x-priority'] = df['x-priority'].astype(str).str.extract('(\\d+)')
        df['x-priority'] = pd.to_numeric(df['x-priority'], errors='coerce')
        df['x-priority'] = df['x-priority'].fillna(0)


        df['x-priority'] = df['x-priority'].apply(lambda x: 0 if x != 3 else 1)

        final_features_list.append('x-priority')

        if 'content-length' in df:

            df['content-length'] = df['content-length'].astype(str).str.extract('(\\d+)')
            df['content-length'] = pd.to_numeric(df['content-length'], errors='coerce')
            df['content-length'] = df['content-length'].fillna(0)

            df[df['content-length'] > 0]['content-length'].describe()

            df['content-length'] = df['content-length'].apply(lambda x: 0 if x < 1 else 1 if x < 1274 else 2 if x < 2348 else 3 if x < 5798 else 4)

        final_features_list.append('content-length')

        if 'lines' in df:

            df['lines'] = df['lines'].astype(str).str.extract('(\\d+)')
            df['lines'] = pd.to_numeric(df['lines'], errors='coerce')

            df['lines'].describe()

            df['lines'] = df['lines'].fillna(0)
            df['lines'] = df['lines'].apply(lambda x: 0 if x == 0 else 1 if x <= 30 else 2 if x <= 54 else 3 if x <= 119 else 4)
            df['lines'].value_counts()

        final_features_list.append('lines')

        if 'last_reveived_date' in df:

            df['last_received_date'] = df['last_received'].str.replace('\n\t', ';').str.split(r';').str[-1]

        if 'first_received_date' in df:
            df['first_received_date'] = df['first_received'].str.replace('\n\t', ';').str.split(r';').str[-1]

        def get_day_week(row):
          date_val = row['date']

          d1 = email.utils.parsedate_tz(date_val)

          if d1 is None:
            return 'NA'

          try:
            val1 = email.utils.mktime_tz(d1)
            day = datetime.fromtimestamp(val1).strftime("%A")
          except:
            return 'NA'

          return day

        df['day_of_week'] = df.apply(get_day_week, axis=1)

        df['day_of_week'] = df['day_of_week'].apply(lambda x: ['NA', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(x))

        final_features_list.append('day_of_week')

        test1 = email.utils.parsedate_tz('Mon, 12 Jul 2021 19:28:19 +0000')
        test2 = email.utils.parsedate_tz('Mon, 12 Jul 2021 19:39:12 +0000')

        print(((email.utils.mktime_tz(test2)) - (email.utils.mktime_tz(test1))) / (60*60))
        print(((email.utils.mktime_tz(test2)) - (email.utils.mktime_tz(test1))) )

        def span_time_finder(row):

          if 'first_received_date' in row:
              first = row['first_received_date']
              d1 = email.utils.parsedate_tz(first)



          if 'last_received_date' in row:

              last = row['last_received_date']
              d2 = email.utils.parsedate_tz(last)

        #   if d1 is None or d2 is None:
        #     return -1

          if 'last_received_date' not in row and 'first_received_date' not in row:
                return -1

          try:
            val1 = email.utils.mktime_tz(d1)
            val2 = email.utils.mktime_tz(d2)
          except:
            return -1




          return (email.utils.mktime_tz(d2)) - (email.utils.mktime_tz(d1))



        df['span_time'] = df.apply(span_time_finder, axis=1)

        df['span_time'] = df['span_time'].apply(lambda x: 0 if x < 0 else 1 if x < 10 else 2 if x < 47 else 3 if x < 1100 else 4)

        # Source: https://github.com/Te-k/pyreceived/blob/master/pyreceived/parser.py

        import re
        class ReceivedParser(object):
            regexes = [
                ("from\s+(mail\s+pickup\s+service|(?P[\[\]\w\.\-]*))\s*(\(\s*\[?(?P[a-f\d\.\:]+)(\%\d+|)\]?\s*\)|)\s*by\s*(?P[\w\.\-]+)\s*(\(\s*\[?(?P[\d\.\:a-f]+)(\%\d+|)\]?\)|)\s*(over\s+TLS\s+secured\s+channel|)\s*with\s*(mapi|Microsoft\s+SMTP\s+Server|Microsoft\s+SMTPSVC(\((?P[\d\.]+)\)|))\s*(\((TLS|version=(?P[\w\.]+)|)\,?\s*(cipher=(?P[\w\_]+)|)\)|)\s*(id\s+(?P[\d\.]+)|)", "MS SMTP Server"), #exchange
                ("(from\s+(?P[\[\S\]]+)\s+\(((?P[\S]*)|)\s*\[(IPv6\:(?P[a-f\d\:]+)\:|)((?P[\d\.\:]+)|)\]\s*(\(may\s+be\s+forged\)|)\)\s*(\(using\s+(?P[\w\.]+)\s+with\s+cipher\s+(?P[\w\-]+)\s+\([\w\/\s]+\)\)\s+(\(No\s+client\s+certificate\s+requested\)|)|)|)\s*(\(Authenticated\s+sender\:\s+(?P[\w\.\-\@]+)\)|)\s*by\s+(?P[\S]+)\s*(\((?P[\S]*)\s*\[((?P[a-f\:\d]+)|)(?P[\d\.]+)\]\)|)\s*(\([^\)]*\)|)\s*(\(Postfix\)|)\s*(with\s+(?P\w*)|)\s*id\s+(?P[\w\-]+)\s*(for\s+\<(?P[\w\.\@]+)\>|)", "postfix"), #postfix
                ("(from\s+(?P[\[\S\]]+)\s+\(((?P[\S]*)|)\s*\[(IPv6\:(?P[a-f\d\:]+)|)\]\)\s*(\(using\s+(?P[\w\.]+)\s+with\s+cipher\s+(?P[\w\-]+)\s+\([\w\/\s]+\)\)\s+(\(No\s+client\s+certificate\s+requested\)|)|)|)\s*(\(Authenticated\s+sender\:\s+(?P[\w\.\-\@]+)\)|)\s*by\s+(?P[\S]+)\s*(\((?P[\S]*)\s*\[((?P[a-f\:\d]+)|)(?P[\d\.]+)\]\)|)\s*(\([^\)]*\)|)\s*(\(Postfix\)|)\s*(with\s+(?P\w+)|)\s*id\s+(?P[\w\-]+)\s*(for\s+\<(?P[\w\.\@]+)\>|)", "postfix"),#POSTFIX
                ("\s*from\s+\[?(?P[\d\.\:]+)\]?\s*(\((port=\d+|)\s*helo=(?P[\[\]\w\.\:\-]+)\)|)\s+by\s+(?P[\w\-\.]+)\s+with\s+(?P\w+)\s*(\((?P[\w\.\:\_\-]+)\)|)\s*(\(Exim\s+(?P[\d\.\_]+)\)|)\s*\(envelope-from\s+(?P[\w\@\-\.]*)>?\s*\)\s*id\s+(?P[\w\-]+)\s*\s*(for\s+(?P[\w\.\@]+)>?|)", "exim"), #exim
                ("\s*from\s+(?P[\w\.]+)\s+\(\[?(?P[\d\.\:a-f]+)\]?(\:\d+|)\s*(helo\=\[?(?P[\w\.\:\-]+)|)\]?\)\s+by\s+(?P[\w\-\.]+)\s+with\s+(?P\w+)\s+(\((?P[\w\.\:\_]+)\)|)\s*\(Exim\s+(?P[\d\.\_]+)\)\s*\(envelope-from\s+\<(?P[\w\@\-\.]+)\>\s*\)\s*id\s+(?P[\w\-]+)\s*(for\s+(?P[\w\.\@]+)|)", "exim"),# exim
                ("from\s+(?P[\w\.\-]+)\s+by\s+(?P[\w\.\-]+)\s+with\s+(?P\w+)\s+\(Exim\s+(?P[\d\.]+)\)\s+\(envelope-from\s+<*(?P[\w\.\-\@]+)>*\)\s+id\s+(?P[\w\.\-]+)\s+for\s+(?P[\w\.\-\@]+)>?", "exim"), #exim
                ("from\s+(?P[\[\]\w\-\.]+)\s+\(((?P[\w\.\-]+)|)\s*\[(?P[\da-f\.\:]+)\]\)\s+by\s+(?P[\w\.\-]+)\s+\(Oracle\s+Communications\s+Messaging\s+Server\s+(?P[\w\.\-]+)(\([\d\.]+\)|)\s+(32bit|64bit|)\s*(\([^\)]+\)|)\)\s*with\s+(?P\w+)\s+id\s+\(?P[\w\@\.\-]+)\>?", "Oracle Communication Messaging Server"), #Oracle
                ("from\s+(?P[\w\-\.]+)\s+\(\[(?P[\d\.\:a-f]+)\]\s+helo=(?P[\w\.\-]+)\)\s+by\s+(?P[\w\.\-]+)\s+with\s+(?P\w+)\s+\(ASSP\s+(?P[\d\.]+)\s*\)", "ASSP"), #ASSP
                ("from\s+(?P[\[\]\d\w\.\-]+)\s+\(\[\[?(?P[\d\.]+)(\:\d+|)\]\s*(helo=(?P[\w\.\-]+)|)\s*\)\s+by\s+(?P[\w\.\-]+)\s+\(envelope-from\s+\(?P[^>]+)\>?\)\s+\(ecelerity\s+(?P[\d\.]+)\s+r\([\w\-\:\.]+\)\)\s+with\s+(?P\w+)\s*(\(cipher=(?P[\w\-\_]+)\)|)\s*id\s+(?P[\.\-\w\/]+)", "ecelerity"), #ecelerity
                ("from\s+(?P[\[\]\w\.\-]+)\s+\(((?P[\w\.\-]+)|)\s*(\[(?P[\d\.\:a-f]+)\]|)\)\s*by\s+(?P[\w\.\-]+)\s+(\([\w\.\-\=]+\)|)\s+with\s+(?P\w+)\s+\(Nemesis\)\s+id\s+(?P[\w\.\-]+)\s*(for\s+\(?P[\w\.\@\-]+)\>?|)", "nemesis"), #nemesis
                ("\(qmail\s+\d+\s+invoked\s+(from\s+network|)(by\s+uid\s+\d+|)\)", "qmail"), #WTF qmail
                ("from\s+\[?(?P[\d\.a-f\:]+)\]?\s+\(account\s+(?P[\w\.\@\-]+)>?\s+HELO\s+(?P[\w\.\-]+)\)\s+by\s+(?P[\w\.\-]*)\s+\(CommuniGate\s+Pro\s+SMTP\s+(?P[\d\.]+)\)\s+with\s+(?P\w+)\s+id\s+(?P[\w\-\.]+)\s+for\s+(?P[\w\.\-\@]+)>?", "CommuniGate"), #CommuniGate
                ("from\s+(?P[\d\.\:a-f]+)\s+\(SquirrelMail\s+authenticated\s+user\s+(?P[\w\@\.\-]+)\)\s+by\s+(?P[\w\.\-]+)\s+with\s+(?P\w+)", "SquirrelMail"),
                ("by\s+(?P[\w\.\-]+)\s+\((?P\w+)\s+sendmail\s*(emulation|)\)", "sendmail"), #sendmail
                ("from\s+(?P[\[\]\w\.\-]+)\s+\(\[(?P[\w\.\-]+)\]\s+\[(?P[\d\.a-f\:]+)\]\)\s+by\s+(?P[\w\.\-]+)\s+\(Sun\s+Java\(tm\)\s+System\s+Messaging\s+Server\s+(?P[\w\.\-]+)\s+\d+bit\s+\(built\s+\w+\s+\d+\s+\d+\)\)\s+with\s+(?P\w+)\s+id\s+(?P[\w\.\-\@]+)>?", "Sun Java System Messaging Server"), # Sun Java System Messaging Server
                ("from\s+(?P[\w\.\-\[\]]+)\s+\((?P[\d\.a-f\:]+)\)\s+by\s+(?P[\w\.\-]+)\s+\(Axigen\)\s+with\s+(?P\w+)\s+id\s+(?P[\w\.\-]+)", "Axigen"), #axigen
                ("from\s+(?P[\w\.\-]+)\s+\((?P[\w\.\-]+)\s+\[(?P[\d\.a-f\:]+)\]\)\s+by\s+(?P[\w\.\-]+)\s+\(Horde\s+MIME\s+library\)\s+with\s+(?P\w+)", "Horde MIME library"), #Horde
                ("from\s+(?P[\w\.\-\[\]]+)\s+by\s+(?P[\w\.\-]+)\s+\(PGP\s+Universal\s+Service\)", "PGP Universal Service", "local"), # PGP Universal Service
                ("from\s+(?P[\w\.\-]+)\s+by\s+(?P[\w\.\-]+)\s+with\s+(?P\w+)\s+\(Sophos\s+PureMessage\s+Version\s+(?P[\d\.\-]+)\)\s+id\s+(?P[\w\.\-]+)\s+for\s+(?P[\w\.\-\@]+)", "Sophos PureMessage"), #Sophos PureMessage
                ("by\s+(?P[\d\.\:a-f]+)\s+with\s+(?P\w+)", "unknown"), # other
                ("from\s+(?P[\w\.\-]+)\s+\#?\s*(\(|\[|\(\[)\s*(?P[\d\.\:a-f]+)\s*(\]|\)|\]\))\s+by\s+(?P[\w\.\-]+)(\s+\([\w\.\s\/]+\)|)\s*(with\s+(?P\w+)|)\s*(id\s+(?P[\w]+)|)(\(\-\)|)\s*(for\s+\<(?P[\w\@\.]+)\>?|)", "unknown"), #unknown
                ("from\s+(?P[\w\.\-]+)\s*\(HELO\s+(?P[\w\.\-]+)\)\s*\(\[?(?P[\d\.\:a-f]+)\]?\)\s+by\s+(?P[\w\.\-]+)(\s+\([\d\.]+\)|)\s*(with\s+(?P\w+)|)\s*(id\s+(?P[\w]+)|)(\(\-\)|)", "unknown"), #other other
                ("from\s+([\(\[](?P[\d\.\:a-f]+)[\)\]]|)\s+by\s+(?P[\w\.\-]+)\s+id\s+(?P\w+)\s*(with\s+(?P\w+)|)\s*\s*(for\s+\<(?P[\w\@\.\-]+)\>|)", "unknown"),#other
                ("from\s+(?P[\w\.]+)\s+(\(HELO\s+(?P[\w\.\-]+)\)|)\s*(\((?P[\da-f\.\:]+)\)|)\s*by\s+(?P[\w\.\-]+)\s+with\s+(?P[\w\-]+)\s+encrypted\s+SMTP", "unknown"), #unknown
                ("from\s+(?P[\w\.\-]+)\s+(\(HELO\s+(?P[\w\.\-]+)\)|)\s+\((?P[\w\.]+\@[\w\.]+)\@(?P[\da-d\.\:]+)\)\s+by\s+(?P[\w\.\-]+)\s+with\s+(?P\w+)", "unknown"), #unknown
                ("from\s+(?P[\w\.\-]+)\s+\(HELO\s+(?P[\w\.\-\?]+)\)\s+\(\w+\@[\w\.]+\@(?P[\d\.a-f\-]+)_\w+\)\s+by\s+(?P[\w\.\-\:]+)\s+with\s+(?P\w+)", "unknown"), #unknown
                ("from\s+(?P[\w\.\-\[\]]+)\s+\(\[(?P[\da-f\.\:]+)\]\)\s+by\s+(?P[\w\.\-]+)\s+\(\[(?P[\d\.a-f\:]+)\]\)\s+with\s+(?P\w+)", "unknown"), #unknown
                ]
            @staticmethod
            def parse(header):
                parts = header.split(";")
                if len(parts) != 2:
                    return None

                data = {}

                # parse the hard part
                found = False
                for regex in ReceivedParser.regexes:
                    match = re.match(regex[0], parts[0], re.IGNORECASE)
                    if match:
                        data['server'] = regex[1]
                        found = True
                        break

                if not found:
                    return None
                return {**data, **match.groupdict()}



        received_parser = ReceivedParser()

        def check_if_valid(dict_to_check, str_val):
          if dict_to_check is None:
            return False
          elif str_val not in dict_to_check:
            return False
          elif dict_to_check[str_val] is None:
            return False
          else:
            return True

        # One-Hot encoding, new columns: conseq_num_received_is_one, conseq_received_good, conseq_received_bad, conseq_received_unknown (All values are 0 or 1)

        def check_conseq_received_domain(row):
          num_received = row['hops']
          if num_received == 1:
            return pd.Series([1,0,0,0])

          col_name_base = 'received'
          for i in range(1, num_received):
            curr_col = col_name_base + str(i)
            next_col = col_name_base + str(i+1)

            curr_val = row[curr_col]
            next_val = row[next_col]

            from_vals_dict = received_parser.parse(curr_val)
            by_val_dict = received_parser.parse(next_val)

            by_valid = check_if_valid(by_val_dict, 'by_hostname')
            from_hostname_valid = check_if_valid(from_vals_dict, 'from_hostname')
            from_name_valid = check_if_valid(from_vals_dict, 'from_name')

            if by_valid:
              # Both are valid case
              if from_hostname_valid and from_name_valid:
                if from_vals_dict['from_name'] == by_val_dict['by_hostname'] or \
                from_vals_dict['from_hostname'] == by_val_dict['by_hostname']:
                  continue
                else:
                  #print("K Val: " + str(i) +"  FROMHOSTNAME: " + from_vals_dict['from_hostname'] + "  FROMNAME: " + from_vals_dict['from_name'] + "   BY: " + by_val_dict['by_hostname'])
                  return pd.Series([0,0,1,0])

              # Neither are valid case
              elif not from_hostname_valid and not from_name_valid:
                # Check my way before saying its invalid
                val = my_checks(curr_val, next_val)
                if isinstance(val, str):
                  continue
                else:
                  return val

              # From hostname is valid case
              elif from_hostname_valid:
                if from_vals_dict['from_hostname'] == by_val_dict['by_hostname']:
                  continue
                else:
                  #print("K Val: " + str(i) +"  FROMHOSTNAME: " + from_vals_dict['from_hostname'] + "   BY: " + by_val_dict['by_hostname'])
                  return pd.Series([0,0,1,0])

              # From name is valid case
              elif from_name_valid:
                if from_vals_dict['from_name'] == by_val_dict['by_hostname']:
                  continue
                else:
                  #print("K Val: " + str(i) + "  FROMNAME: " + from_vals_dict['from_name'] + "   BY: " + by_val_dict['by_hostname'])
                  return pd.Series([0,0,1,0])
            else:
              # Check my way before saying its invalid
              val = my_checks(curr_val, next_val)
              if isinstance(val, str):
                continue
              else:
                return val

          # All checks worked out, return a good result
          return pd.Series([0,1,0,0])


        def my_checks(curr_val, next_val):
          first_domain_from = re.search(r'((?<=\bfrom\s)[^\s]+)', curr_val)
          second_domain_from = re.search(r'((?<=\().*?(?=\[))', curr_val)
          domain_by = re.search(r'((?<=\bby\s)[^\s]+)', next_val)

          if domain_by is not None:
            # Both are valid case
            if first_domain_from is not None and second_domain_from is not None:
                if first_domain_from.group(0) == domain_by.group(0) or \
                second_domain_from.group(0) == domain_by.group(0):
                  return 'continue'
                else:
                  return pd.Series([0,0,1,0])

            # Neither are valid case
            if first_domain_from is None and second_domain_from is None:
              return pd.Series([0,0,0,1]) 

            # One is valid case
            if first_domain_from is not None:
              if first_domain_from.group(0) == domain_by.group(0):
                return 'continue'
              else:
                return pd.Series([0,0,1,0])

            # One is valid case
            if second_domain_from is not None:
              if second_domain_from.group(0) == domain_by.group(0):
                return 'continue'
              else:
                return pd.Series([0,0,1,0])

          else:
            return pd.Series([0,0,0,1])


        from difflib import SequenceMatcher

        def simScore(a, b):
          if isinstance(a, float) or isinstance(b, float):
            return 0
          return SequenceMatcher(None, a, b).ratio()

        # emails in brackets '<>' are matched first, and if none, then other emails are matched
        def extract_emails(row, col_name):

          in_brackets = re.findall(r'<([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)>', row[col_name])

          if len(in_brackets) == 0:
            not_in_brackets = re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', row[col_name])
            if len(not_in_brackets) == 0:
              return []
            else:
              return not_in_brackets
          else:
            return in_brackets



        newflag = 0
        newflag1 = 0
        newflag2 = 0
        newflag3 = 0
        newflag4 =0 
        newflag5 = 0
        newflag6 = 0
        newflag7 = 0
        newflag8 = 0
        newflag9 = 0

        if 'from' in df:
            emails_from = df.apply(extract_emails, col_name='from', axis=1)
            newflag1 = 1

        if 'message-id' in df:
            emails_message_id = df.apply(extract_emails, col_name='message-id', axis=1)
            newflag2 = 1

        if 'return-path' in df:
            emails_return_path = df.apply(extract_emails, col_name='return-path', axis=1)
            newflag3 = 1

        if 'reply-to' in df:
            emails_reply_to = df.apply(extract_emails, col_name='reply-to', axis=1)
            newflag4 = 1

        if 'errors-to' in df:
            emails_errors_to = df.apply(extract_emails, col_name='errors-to', axis=1)
            newflag = 1

        if 'in-reply-to' in df:
            emails_in_reply_to = df.apply(extract_emails, col_name='in-reply-to', axis=1)
            newflag5 = 1

        if 'references' in df:
            emails_references = df.apply(extract_emails, col_name='references', axis=1)
            newflag6 = 1

        if 'to' in df:
            emails_to = df.apply(extract_emails, col_name='to', axis=1)
            newflag6 = 1

        if 'cc' in df:
            emails_cc = df.apply(extract_emails, col_name='cc', axis=1)
            newflag7 = 1

        if 'sender' in df:
            emails_sender = df.apply(extract_emails, col_name='sender', axis=1)
            newflag8 = 1

        #simScores = domains_df[['return', 'from']].apply(lambda x: simScore(*x), axis=1)
        #df['SimScore_return_from'] = simScores


        if newflag1==1:
            if newflag2 == 1:

                if newflag3 == 1:

                    if newflag4==1:

                        if newflag5==1:

                            if newflag6 == 1:


                                if newflag7==1:

                                    if newflag8==1:

                                        if newflag == 1:

                                            emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                                    emails_errors_to, emails_reply_to, emails_in_reply_to, 
                                                                    emails_references, emails_to, emails_cc, emails_sender], axis=1)
                                            emails_df.set_axis(['from', 'message-id', 'return-path', 'errors-to', 'reply-to',
                                                                 'in-reply-to', 'references', 'to', 'cc', 'sender'], 
                                                                axis=1, inplace=True)
                                        else:
                                            emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                                     emails_reply_to, emails_in_reply_to, 
                                                                    emails_references, emails_to, emails_cc, emails_sender], axis=1)
                                            emails_df.set_axis(['from', 'message-id', 'return-path', 'reply-to',
                                                                 'in-reply-to', 'references', 'to', 'cc', 'sender'], 
                                                                axis=1, inplace=True)

                                    else:
                                        emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                                emails_errors_to, emails_reply_to, emails_in_reply_to, 
                                                                emails_references, emails_to, emails_cc], axis=1)
                                        emails_df.set_axis(['from', 'message-id', 'return-path', 'reply-to',
                                                             'in-reply-to', 'references', 'to', 'cc'], 
                                                            axis=1, inplace=True)
                                else:
                                    emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                             emails_reply_to, emails_in_reply_to, 
                                                            emails_references, emails_to], axis=1)
                                    emails_df.set_axis(['from', 'message-id', 'return-path', 'reply-to',
                                                         'in-reply-to', 'references', 'to'], 
                                                        axis=1, inplace=True)

                            else:
                                emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                             emails_reply_to, emails_in_reply_to, 
                                                             emails_to], axis=1)
                                emails_df.set_axis(['from', 'message-id', 'return-path', 'reply-to',
                                                         'in-reply-to', 'to'], 
                                                        axis=1, inplace=True)

                        else:
                            emails_df = pd.concat([emails_from, emails_message_id, emails_return_path, 
                                                           emails_reply_to, 
                                                             emails_to], axis=1)
                            emails_df.set_axis(['from', 'message-id', 'return-path', 'reply-to',
                                                          'to'], 
                                                        axis=1, inplace=True)

                    else:
                        emails_df = pd.concat([emails_from, emails_message_id, emails_return_path,

                                                             emails_to], axis=1)
                        emails_df.set_axis(['from', 'message-id','return-path',
                                                          'to'], 
                                                        axis=1, inplace=True)

                else:
                    emails_df = pd.concat([emails_from,emails_message_id, emails_to], axis=1)
                    emails_df.set_axis(['from', 'message-id','to'], 
                                                        axis=1, inplace=True)
            else:
                emails_df = pd.concat([emails_from, emails_to], axis=1)
                emails_df.set_axis(['from', 'to'], 
                                                        axis=1, inplace=True)


        else:
            emails_df = pd.concat([], axis=1)
            emails_df.set_axis([], 
                                axis=1, inplace=True)


        parser = ReceivedParser()
        test = parser.parse('from cnnimail22.cnn.com (cnnimail22.cnn.com [64.236.25.79])\n\tby speedy.uwaterloo.ca (8.12.8/8.12.5) with ESMTP id l39IYQ0I018124\n\tfor  Mon, 9 Apr 2007 14:34:26 -0400')

        def get_for_domain_last_received(row):
          last_received_val = row['last_received']
          parsed_val = parser.parse(last_received_val)

          if check_if_valid(parsed_val, 'envelope_for'):
            main_domain = parsed_val['envelope_for'].split('@')[-1]
            main_domain = main_domain.split('.')[-2:]
            main_domain = main_domain[0] + '.' + re.sub('\W+','', main_domain[1])
            return main_domain.lower()
          else:
            return 'NA'

        def check_for_received_domain_equal(row, field_name):
          if field_name in row:
              field_vals = row[field_name]

              for item in field_vals:
                if item == get_for_domain_last_received(row):
                  return 1
          return 0

        df['domain_match_to_received'] = df.apply(check_for_received_domain_equal, field_name='to_domains', axis=1)
        df['domain_match_to_received'].value_counts()



        for item in final_features_list:

          if item in df:  
              print(item)

              print(df[item].value_counts())
              print('\n')


        df[df['str_to_chevron'] == 0]['label'].value_counts()


        # Removes missing_received fields since received1 is always there, and the other ones
        # encode the same information as 'hops'. The other removed features have only one value,
        # or a strong majority towards one value.
        remove_list = ['missing_received1', 'missing_received2', 'missing_received3',
         'missing_received4', 'missing_received5', 'missing_received6',
         'missing_received7', 'missing_received8', 'missing_received9',
         'missing_received10', 'missing_received11', 'missing_received12',
         'missing_received13', 'missing_received14', 'missing_received15',
         'missing_received16', 'missing_date', 'missing_message-id', 'missing_from',
         'missing_return-path', 'str_to_undisclosed', 'str_return-path_empty',
         'str_from_exclam', 'str_reply-to_question', 'str_received-SPF_bad', 
         'str_received-SPF_softfail', 'str_received-SPF_fail', 'str_reply-to_question', 
         'conseq_received_unknown', 'num_recipients_from', 
         'domain_match_reply-to_received', 'domain_match_return-path_received',
         'domain_match_from_received']

        for v in remove_list:
          if v in final_features_list:
            final_features_list.remove(v)

        final_features_list.append('label')




        # final_fet = final_features_list

        # for item in final_fet:
        #     if item not in df:
        #         final_fet.remove(item)

        # df_final = df[final_fet]

        for item in final_features_list:
            if item not in df:
                df[item] = 0



        df_final = df[final_features_list]


        rand_list = [0,1]

        for item in final_features_list:
            if item not in df:
                df[item] = int(random.choices(rand_list, weights=(55,45), k=2))



        df_final = df[final_features_list]

        df_final.to_csv('preprocessed_spam_ham_new.csv', index=False)




        inp_df = pd.read_csv("preprocessed_spam_ham_new.csv")

        df_row = inp_df.iloc[0:1]

        df_row = df_row.iloc[: , :45]


        pred = clf.predict(df_row)
    
    except:
        print("Ignored")
        return -1
    
    return pred

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

vizer =  TfidfVectorizer(stop_words='english')

In [6]:
def ContentDetectionTrain(content):

    df = pd.read_csv("spam.csv")

    df.head()
    
    
    
    df.groupby('Category').describe()
    
    df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)

    df.head()
    
    


    X_train,X_test,Y_train,Y_test = train_test_split(df.Message,df.spam,test_size=0.20)


    X_train.describe()
    
    X_test.describe()
    

    


    X_train_count= vizer.fit_transform(X_train.values)


    print(X_train_count.toarray())
    
    


    model = MultinomialNB()


    model.fit(X_train_count,Y_train)
    emails = []
    emails.append(content)
    
    emails_count = vizer.transform(emails)
    
    res=model.predict(emails_count)

    return res[0]
    

In [7]:
from PIL import Image
import numpy as np
import pickle

In [8]:
def ImageDetectionInput(img_path,modl='./model.pkl'):
    model = pickle.load(open(modl,'rb'))
    try:
        img = Image.open(img_path)
        img = img.resize((200,200))
        imr = np.array(img)/255
        imr = imr.reshape([200,200,3])
        
    except Exception as e:
        print("Error")
        return -1
    
    imr = imr.reshape(1,200,200,3)
    prediction = model.prediction(imr)
    return 0 if prediction < 0.5 else 1

In [9]:
HeaderTrain()


c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 

Accuracy: 99.68708565367277
F1 Score: 99.76531112849547
Recall: 99.87787718692755
Precision: 99.65299851663488
ROC AUC: 99.5923410699163
Confusion Matrix: [[18767   131]
 [   46 37621]]
-----------------------------------------



c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\adhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 

Accuracy: 99.69415716432422
F1 Score: 99.77044438252193
Recall: 99.808851249104
Precision: 99.73206706281834
ROC AUC: 99.63720157967953
Confusion Matrix: [[18797   101]
 [   72 37595]]
-----------------------------------------



In [10]:
headerfile = "headertext.txt"

In [11]:
content = "Wow. I never realized that you were so embarrassed by your accommodations. I thought you liked it, since I was doing the best I could and you always seemed so happy about \"the cave\". I'm sorry I didn't and don't have more to give. I'm sorry I offered. I'm sorry your room was so embarrassing."

In [12]:
img_path = 'zzz_0969_d70494b85b_m.jpg'

In [13]:
pred1 = HeaderInput(headerfile)
pred2 = ContentDetectionTrain(content)



Ignored
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# pred3 = ImageDetectionInput(img_path)

In [15]:
if pred1==-1:
    if pred2 == 1:
        print("The mail is Spam")
    else:
        print("The mail is Ham")
        
# elif pred1 == 1:
#     if pred2 == 1:
#         print("The mail is Spam")
#     else:
#         print("The mail is Ham")
# else:
#     if pred2 == 1:
#         print("The mail is Spam")
#     else:
#         print("The mail is Ham")
    


if pred1 ==1 or pred2 == 1:
    print("The mail is Spam")
else:
    print("The mail is Ham")
    

The mail is Ham
The mail is Ham


In [16]:
# import statistics



# def final_pred(p1,p2,p3):
    
#     final_ans = statistics.mode(p1,p2,p2)
    
#     return final_ans

In [17]:
# final_prediction = final_pred(pred1,pred2,pred3)

# if final_prediction == 1:
#     print("The Mail is Spam")
    
# else:
#     print("The Mail is Ham")